In [1]:
import torch

import ncg_optimizer as optim

In [2]:
def quadratic(tensor):
    x, y = tensor
    a = 2.0
    b = 1.0
    return (x ** 2) / a + (y ** 2) / b + 2 * x * y

In [3]:
def rosenbrock(tensor):
    x, y = tensor
    return (1 - x) ** 2 + 1 * (y - x ** 2) ** 2

In [4]:
def beale(tensor):
    x, y = tensor
    f = (
        (1.5 - x + x * y) ** 2
        + (2.25 - x + x * y ** 2) ** 2
        + (2.625 - x + x * y ** 3) ** 2
    )
    return f

In [5]:
initial_state = (1.5, 1.5)
min_loc = (0, 0)
x = torch.Tensor(initial_state).requires_grad_(True)
x_min = torch.Tensor(min_loc)
iterations = 500

In [6]:
optimizer = optim.LCG([x], eps=1e-3)
for _ in range(iterations):
    optimizer.zero_grad()
    f = quadratic(x)
    f.backward(retain_graph=True, create_graph=True)
    optimizer.step()
torch.allclose(x, x_min, atol=0.001)

True